In [1]:
import math 
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

In [2]:
def bin_space_gen(d):
    all_coeff = {}
    for i in range(0,2**(d)): 
        t = i
        all_coeff[i] = ""
        for j in range (0,d):
            all_coeff[i]= str(t%2)+all_coeff[i]
            t = int(t/2)
    return all_coeff

In [3]:
def ter_space_gen(d):
    all_coeff = {}
    for i in range(0,3**(d)): 
        t = i
        all_coeff[i] = ""
        for j in range (0,d):
            all_coeff[i]= str(t%3)+all_coeff[i]
            t = int(t/3)
    return all_coeff

def ter_dec(y):
    y_dec = 0
    for i in range(len(y)):
        if(y[i]=="1"):
            y_dec += 3**(len(y)-i-1)
        if(y[i]=="*" or y[i]=="2"):
            y_dec += 2*3**(len(y)-i-1)
    return y_dec
    

In [4]:
def choose(n, k):
    """
    A fast way to calculate binomial coefficients by Andrew Dalke (contrib).
    """
    if 0 <= k <= n:
        ntok = 1
        ktok = 1
        for t in range(1, min(k, n - k) + 1):
            ntok *= n
            ktok *= t
            n -= 1
        return ntok // ktok
    else:
        return 0

In [5]:
def BSC(x,p):
  y = ""
  for i in range(len(x)):
    if np.random.binomial(1, p):
      y += str((int(x[i])+1)%2)
    else:
      y += x[i]
  return y

def BEC(x,p):
  y = ""
  for i in range(len(x)):
    if np.random.binomial(1, p):
      y += "*"
    else:
      y += x[i]
  return y

def gen_data(C,L):
    D = {}
    for i in range(len(C)):
        d = ""
        for _ in range(L):
            d += random.choice(("0","1"))
        D[i] = (C[i],d)
    return D

def ers_chan(N,D,p):
    Y = {}
    IO = {}
    for i in range(len(D)): 
        for j in range(N):
            y = BEC(D[i][0],p) 
            d_dash = BEC(D[i][1],p)
            Y[N*i+j] = (y,d_dash)
            IO[N*i+j] = (D[i],Y[N*i+j])
    return Y,IO

In [6]:
def bin_dec(y):
  y_dec = 0
  for i in range(len(y)):
    if(y[i]=="1"):
      y_dec += 2**(len(y)-i-1)
  return y_dec
  
def dec_bin(y):
  y_bin = ""
  while(y>0):
    y_bin = str(y%2) + y_bin
    y = int(y/2)
  return y_bin

def pos_inps(y):
  inps = []
  ers = []
  for i in range(len(y)):
    if(y[i]=="*"):
      ers.append(i)
  ers_rep = bin_space_gen(len(ers))
  for i in range(len(ers_rep)):
    inp = list(y)
    for j in range(len(ers)):
      inp[ers[j]] = ers_rep[i][j]
    inp_str = ""
    for j in range(len(inp)):
      inp_str += inp[j]
    inps.append(inp_str)
  
  return inps

def buil_graph_out(IO):
    graph_out = {} 
    for i in range(len(IO)):
        y = IO[i][1][0]
        edges = pos_inps(y)
        graph_out[i] = (IO[i][1], edges)
    return graph_out

In [7]:
def build_graph_in(graph_out,C):
    graph_in = {}
    for i in range(len(C)):
        graph_in[i] = (C[i],[])

    for i in range(len(graph_out)):
        for j in range(len(graph_out[i][1])):
            graph_in[bin_dec(graph_out[i][1][j])][1].append(graph_out[i][0])
  
    return graph_in

In [8]:
def pel_dec(graph_in_org,N,n,CNT):
    cnt = 0
    gr = graph_in_org
    mark_out = np.zeros(3**n)
    mark_in = np.zeros(2**n)
    
    for i in range(len(gr)):
        if(len(gr[i][1])==N):
            mark_in[bin_dec(gr[i][0])] = 1
            for j in range(N):
                mark_out[ter_dec(gr[i][1][j][0])] = 1 
    
    for _ in range(CNT):
#         print(cnt)
        for i in range(len(gr)):
            if(mark_in[bin_dec(gr[i][0])] == 0):
                j = 0 
                while j < len(gr[i][1]):
                    if(mark_out[ter_dec(gr[i][1][j][0])]==1):
                        cnt +=1 
                        del gr[i][1][j]
                        j = j-1
                    j = j+1
                if(len(gr[i][1])==N):
                    mark_in[bin_dec(gr[i][0])] = 1
                    for j in range(N):
                        mark_out[ter_dec(gr[i][1][j][0])] = 1

    return gr

In [9]:
n = 3
p = 0.12
C = bin_space_gen(n)
N = 3
L = 20
D = gen_data(C,L)
Y,IO = ers_chan(N,D,p)

graph_out_org = buil_graph_out(IO)
graph_in_org = build_graph_in(graph_out_org,C)
# graph_in_org

In [10]:
graph_in_org

{0: ('000',
  [('000', '01001**0111*0101*000'),
   ('00*', '*10010*0*1*001010000'),
   ('0*0', '*1001000111001010000'),
   ('00*', '0010111**00***1*0101'),
   ('***', '100011010000111*0101'),
   ('*00', '111001011*0000000011')]),
 1: ('001',
  [('00*', '*10010*0*1*001010000'),
   ('*01', '001011111001*1*10101'),
   ('00*', '0010111**00***1*0101'),
   ('001', '0010*11110*101110101'),
   ('***', '100011010000111*0101'),
   ('*01', '1001**011110*0110101')]),
 2: ('010',
  [('0*0', '*1001000111001010000'),
   ('***', '100011010000111*0101'),
   ('010', '10001101000011100101'),
   ('010', '10*01101000011100101'),
   ('*10', '01110100010*100000*1'),
   ('*1*', '011*01*0*10010000011')]),
 3: ('011',
  [('***', '100011010000111*0101'),
   ('*11', '0001110*010110100101'),
   ('011', '00011*01010*1*100101'),
   ('011', '*001*101010110100101'),
   ('*1*', '011*01*0*10010000011'),
   ('*11', '10110011110001100011')]),
 4: ('100',
  [('***', '100011010000111*0101'),
   ('*00', '111001011*0000000011

In [11]:
pel_dec(copy.deepcopy(graph_in_org),N,n,5) 

{0: ('000',
  [('000', '01001**0111*0101*000'),
   ('00*', '*10010*0*1*001010000'),
   ('0*0', '*1001000111001010000'),
   ('00*', '0010111**00***1*0101'),
   ('***', '100011010000111*0101'),
   ('*00', '111001011*0000000011')]),
 1: ('001',
  [('00*', '*10010*0*1*001010000'),
   ('*01', '001011111001*1*10101'),
   ('00*', '0010111**00***1*0101'),
   ('001', '0010*11110*101110101'),
   ('***', '100011010000111*0101'),
   ('*01', '1001**011110*0110101')]),
 2: ('010',
  [('0*0', '*1001000111001010000'),
   ('***', '100011010000111*0101'),
   ('010', '10001101000011100101'),
   ('010', '10*01101000011100101'),
   ('*10', '01110100010*100000*1'),
   ('*1*', '011*01*0*10010000011')]),
 3: ('011',
  [('***', '100011010000111*0101'),
   ('*11', '0001110*010110100101'),
   ('011', '00011*01010*1*100101'),
   ('011', '*001*101010110100101'),
   ('*1*', '011*01*0*10010000011'),
   ('*11', '10110011110001100011')]),
 4: ('100',
  [('***', '100011010000111*0101'),
   ('*00', '111001011*0000000011

In [12]:
def valid_hash(bucket,n,L):
    for i in range(n):
        ind = [0,0]
        for j in range(len(bucket)):
            if bucket[j][0][i] == "*":
                continue
            else:
                ind[int(bucket[j][0][i])] += 1 
        if (ind[0]!=0 and ind[1]!=0):
            return 0
        
    for i in range(L):
        data = [0,0]
        for j in range(len(bucket)):
            if bucket[j][1][i] == "*":
                continue
            else:
                data[int(bucket[j][1][i])] += 1 
        if (data[0]!=0 and data[1]!=0):
            return 0
        
    return 1 
            

def hash_func(Y,l,N):
    ht = {}
    hash_table = bin_space_gen(l)
    for i in range(2**l):
        temp = hash_table[i]
        hash_table[i] = (temp,[])
    for i in range(len(Y)):
        d_dash = Y[i][1]  
        edges = pos_inps(d_dash[0:l])
        for j in range(len(edges)):
            hash_table[bin_dec(edges[j])][1].append(Y[i])
    for i in range(2**l):
        if len(hash_table[i][1]) == N:
            if (valid_hash(hash_table[i][1],n,L)):
                ht[i] = hash_table[i]
    
    return ht

In [13]:
l = 10
hash_func(Y,l,N)

{117: ('0001110101',
  [('*11', '0001110*010110100101'),
   ('011', '00011*01010*1*100101'),
   ('011', '*001*101010110100101')]),
 190: ('0010111110',
  [('*01', '001011111001*1*10101'),
   ('00*', '0010111**00***1*0101'),
   ('001', '0010*11110*101110101')]),
 291: ('0100100011',
  [('000', '01001**0111*0101*000'),
   ('00*', '*10010*0*1*001010000'),
   ('0*0', '*1001000111001010000')]),
 465: ('0111010001',
  [('110', '011101*0010010000011'),
   ('*10', '01110100010*100000*1'),
   ('*1*', '011*01*0*10010000011')]),
 564: ('1000110100',
  [('***', '100011010000111*0101'),
   ('010', '10001101000011100101'),
   ('010', '10*01101000011100101')]),
 615: ('1001100111',
  [('101', '1001*0011110101*010*'),
   ('*01', '1001**011110*0110101'),
   ('101', '100110011*101011010*')]),
 719: ('1011001111',
  [('*11', '10110011110001100011'),
   ('111', '10110*111100*1*00***'),
   ('111', '1011001*11000*100011')]),
 919: ('1110010111',
  [('*00', '111001011*0000000011'),
   ('1*0', '*11*01011100*0

In [35]:
for l in range(n, int(L/2)):
    print(len(hash_func(Y,l,N)))

2
3
5
10
13
18


In [20]:
N = 7
p = 0.05
for n in range(10,40):
    print(1-(1-(1-p)**n)**N, 1/(math.log(1/(1-(1-p)**n),2)) )

0.998325040652553 0.7590825690084839
0.9972282719643528 0.824014263819702
0.9956656484607357 0.8917233670802086
0.9935350294829077 0.9624453158687106
0.9907355644369884 1.036410399952006
0.9871716487679206 1.113847918495498
0.9827563790520708 1.1949892185943445
0.9774143673704058 1.2800700184758156
0.9710838483779504 1.3693322622910884
0.9637180719484774 1.4630256632745617
0.9552860190147093 1.5614090379900387
0.9457725090496361 1.664751500915434
0.9351777863202411 1.7733335673162387
0.923516680792816 1.8874481984684282
0.9108174406446605 2.0074018140419767
0.8971203288491626 2.13351529018328
0.8824760680842463 2.266124957514187
0.8669442077802614 2.4055836102512265
0.8505914756610515 2.5522615355253637
0.833490164531038 2.7065475704773942
0.8157165939520378 2.868850193634798
0.7973496762512601 3.039598656318958
0.7784696072501671 3.21924415930217
0.7591566943151188 3.4082610795723687
0.739490327821219 3.6071482518284013
0.7195480968397208 3.8164303091898586
0.6994050457123304 4.036659

In [108]:
N = 5
p = 0.2
for n in range(10,40):
    print(1-(1-(1-p)**n)**N, 2**(N*(2**n)*math.log((2**n-(1+p)**n)/(2**n-1),2)))

0.43330782041120397 4.858867207598076e-12
0.3617804572180058 1.0196303071354215e-14
0.299508933157436 6.159824767659326e-18
0.24627112669033757 8.377997890325009e-22
0.2013916756881663 1.908419355135041e-26
0.16397041292315429 5.10170778791772e-32
0.1330345586734899 1.044042617877196e-38
0.10763228833793514 9.792077388816843e-47
0.0868847424119985 2.2735630374639596e-56
0.07001039179012736 6.239651008870819e-68
0.05633208405131673 8.335104828928271e-82
0.0452739645258623 1.8686527028814744e-98
0.03635303890594144 1.9591398276829794e-118
0.029168392191232018 2.0728245831712145e-142
0.0233898756010118 3.5144158234489847e-171
0.018747279343393042 1.0493625404763664e-205
0.015020504545770752 3.8990384294511636e-247
0.01203093930762189 7.498187239468157e-297
0.00963406441509218 0.0
0.0077132170275164835 0.0
0.006174394200654132 0.0
0.004941961894215474 0.0
0.003955135995240555 0.0
0.003165111700948131 0.0
0.002532731402958177 0.0
0.002026596123168245 0.0
0.0016215399870780356 0.0
0.00129740

In [120]:
N = 5
p = 0.3

for n in range(10,40):
    L = 3*n
    d = L/2 - (2*p - p**2)

    print( (1-((1-((1-p)**2))**d))**(N*((1+p)**n - 1)))

0.09035630888927688
0.10276465302520364
0.11677983874116964
0.1323356583257744
0.14935118745553178
0.16772923378321197
0.18735725965000205
0.2081097190389702
0.22985111813974093
0.2524393381257491
0.2757289148668456
0.2995740818430472
0.3238314631098223
0.3483623606430494
0.37303462026491896
0.39772408691929423
0.42231567680555604
0.4467041035329982
0.4707943001616097
0.4945015803141635
0.5177515806370412
0.5404800245537579
0.5626323440895854
0.5841631928810845
0.6050358796916621
0.6252217478690217
0.644699522539848
0.6634546437558386
0.6814786006353738
0.6987682785815055


In [109]:
N = 6
L = 40
p = 0.3
d = L/2 - (2*p - p**2)
for n in range(10,40):
    print(1-(1-(1-p)**n)**N, (1-((1-((1-p)**2))**d))**(N*((1+p)**n - 1)))

0.15795765345241097 0.9998467534470277
0.11292721795273764 0.9997971888168362
0.08022649184278474 0.9997327584708275
0.05674334339144682 0.9996490052283831
0.040009612383631654 0.999540136502756
0.02814941148751815 0.9993986248848066
0.019774826430778547 0.9992146897330013
0.013876906751393414 0.9989756246442077
0.009730791881659595 0.998664925536296
0.006819876486459431 0.998261161157739
0.004777995923573752 0.9977365115049527
0.00334659906583612 0.9970548791610596
0.0023436008190700353 0.9961694532531066
0.0016410016778347636 0.9950195749790447
0.0011489369783731496 0.993526717313666
0.0008043714502742327 0.9915893503158566
0.0005631166496227191 0.9890764199966912
0.00039420940813739147 0.9858191288975301
0.00027596018573150527 0.981600682234379
0.0001931787943268981 0.9761436766718923
0.00013522842164548354 0.9690948981542795
9.466149533876589e-05 0.9600075258978108
6.626383084107346e-05 0.9483212170552303
4.638506580378898e-05 0.9333414334774057
3.246973432891043e-05 0.914220904774

In [13]:
N = 5
# d = 2 
p = 0.2
for n in range(10,40):
    print(n)
    for d in range(1,5):
        print(1-(1-(1-(p**d))**n)**N)


10
0.43330782041120397
0.9957703092159234
0.9999972613255144
0.999999998988477
11
0.3617804572180058
0.9938040665644324
0.9999956758018178
0.9999999983774224
12
0.299508933157436
0.9912866977774261
0.9999934496906235
0.99999999750302
13
0.24627112669033757
0.988158015103329
0.9999904170895191
0.9999999962889841
14
0.2013916756881663
0.9843657712604313
0.999986390016348
0.9999999946459314
15
0.16397041292315429
0.9798665682577233
0.9999811582106957
0.9999999924704641
16
0.1330345586734899
0.9746263683937484
0.9999744890959266
0.9999999896442765
17
0.10763228833793514
0.9686206638136433
0.999966127886426
0.9999999860332864
18
0.0868847424119985
0.9618343624227621
0.9999557978257084
0.9999999814867913
19
0.07001039179012736
0.9542614459888863
0.9999432005420128
0.9999999758366475
20
0.05633208405131673
0.9459044521860521
0.9999280165089116
0.9999999688964727
21
0.0452739645258623
0.9367738270589963
0.9999099055993089
0.9999999604608726
22
0.03635303890594144
0.9268871885674169
0.999888507

In [27]:
bin_dec(C[10])

10

In [60]:
def does_agree(a,b):
    for i in range(len(a)):
        if a[i] == "*" or b[i] == "*":
            continue 
        else :
            if a[i]!=b[i]:
                return 0
    return 1 

In [106]:
n = 10
p = 0.3
C = bin_space_gen(n)
N = 5
L = 40
D = gen_data(C,L)
Y,IO = ers_chan(N,D,p)
graph_out_org = buil_graph_out(IO)
graph_in_org = build_graph_in(graph_out_org,C)

G = {}
for i in range(len(graph_out_org)):
    if(len(graph_out_org[i][1])==1):
        G[bin_dec(graph_out_org[i][1][0])] =  graph_out_org[i][0]
        
CNT = 0
for i in G.keys():
    cnt = 0 
    E_x = graph_in_org[bin_dec(G[i][0])]
    y_g = G[i][1]
    for j in range(len(E_x[1])):
        cnt += does_agree(y_g,E_x[1][j][1])
    if cnt == N:
        CNT += 1    
        
CNT/len(G)

1.0

23 23


In [16]:
N = 6
p = 0.1

# Sufficiently large n for the approximation to hold
print(3*math.log(N,2)/(math.log(1/(1-p),2)))

for n in range(20,50):
    print(n,":",(1-(1-(1-p)**n)**N), (N*(1-p)**n))

51.01795842705597
20 : 0.5405659110595672 0.7294599275434162
21 : 0.501068889715288 0.6565139347890745
22 : 0.46314054551582573 0.5908625413101671
23 : 0.4269795242602452 0.5317762871791504
24 : 0.392720813965621 0.47859865846123534
25 : 0.3604448173561069 0.43073879261511183
26 : 0.330186255750362 0.38766491335360065
27 : 0.30194252773233166 0.34889842201824056
28 : 0.27568129966829713 0.31400857981641656
29 : 0.25134721867408694 0.2826077218347749
30 : 0.22886771896911418 0.25434694965129745
31 : 0.20815794656638098 0.2289122546861677
32 : 0.18912486115125204 0.2060210292175509
33 : 0.17167059306306764 0.18541892629579582
34 : 0.15569514178287913 0.16687703366621626
35 : 0.14109850358946974 0.15018933029959464
36 : 0.12778231260941386 0.13517039726963517
37 : 0.11565107322321722 0.12165335754267165
38 : 0.10461305403423182 0.1094880217884045
39 : 0.09458090528370688 0.09853921960956405
40 : 0.08547205331705066 0.08868529764860765
41 : 0.0772089178576062 0.07981676788374688
42 : 0.069